# Pizza Bot

## Initialize ChatGPT with your personal key

In [2]:
from openai import OpenAI
from key import openai_key, wit_key

client = OpenAI(
    api_key=openai_key,
)

## Create a prompt for an automated LLM service

In [3]:
context = [ {'role':'system', 'content':"""
You are an OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collect the order and then ask if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.\
You respond in a short, very conversational friendly style. \
Don't allow orders for things not on the menu. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

This method sends the current dialog to ChatGPT and returns the response

In [4]:

def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0):
    response = client.chat.completions.create(model=model, messages=messages, temperature=temperature)
    return response.choices[0].message.content

This method appends the new user prompt to the context, gets a response and appends that to the context

In [5]:
def collect_messages(prompt):
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    return response

## Start Conversation

In [ ]:
human_response = None
while human_response != 'done':
    gpt_response   = collect_messages(human_response)
    print(gpt_response)
    human_response = input(f"\n{gpt_response}\n\n")
    print(human_response)

## Text 2 Speech

In [6]:
from pathlib import Path

i = 2
speech_file_path = Path(f"speech{i}.wav")
print(speech_file_path)

response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="Today is a wonderful day to build something people love!"
)

# The following method is depricated and gives a warning but still seems to work fine.
#response.stream_to_file(speech_file_path)
# This does the same thing without the warning
response.write_to_file(speech_file_path)

speech2.wav


## Speech 2 Text

In [7]:
import speech_recognition as sr
from os import path
import librosa
import soundfile as sf
from timer import Timer

def wav2text(speech_file_path, service):
    AUDIO_FILE = path.join('.', speech_file_path)

    # Sometimes audio files don't have the necessary headers, this will add them
    x,_ = librosa.load(AUDIO_FILE, sr=16000)
    sf.write(AUDIO_FILE, x, 16000)

    r = sr.Recognizer()
    with sr.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source)  # read the entire audio file

    with Timer(service):
        if service == 'Sphinx':
            text = r.recognize_sphinx(audio)
        elif service =='GoogleSpeech':
            text = r.recognize_google(audio)
        elif service =='GoogleCloudSpeech':
            # You will need a Google Cloud account, enable the Cloud Speech-to-Text API
            # and create a credentials file to use this service. Get started by following
            # these instructions https://cloud.google.com/iam/docs/keys-create-delete#python
            text = r.recognize_google_cloud(audio, 'google_cloud_credentials.json')
        elif service =='Wit':
            # You will need to create an account at https://wit.ai/ and generate 
            # a 'Client Access Token' for the key argument
            text = r.recognize_wit(audio, key=wit_key)

            #with open('test.wav', 'rb') as f:
            #text = client.speech(audio, {'Content-Type': 'audio/wav'})

    # recognize speech using Sphinx
    try:
        print(f"{service} thinks you said {text}")
    except sr.UnknownValueError:
        print(f"{service} could not understand audio")
    except sr.RequestError as e:
        print(f"{service} error; {0}".format(e))

In [ ]:
speech_file_path = 'speech2.wav'
for _ in range(10):
    wav2text(speech_file_path, 'Sphinx')
    wav2text(speech_file_path, 'GoogleSpeech')
    wav2text(speech_file_path, 'GoogleCloudSpeech')
    wav2text(speech_file_path, 'Wit')

Timer().report()


## Have the conversation in gradio

In [8]:
def collect_messages(message, history):
    context.append({'role':'user', 'content':f"{message}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    return response

In [10]:
%load_ext gradio

c:\Users\holes\.conda\envs\ik\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
%%blocks

import gradio as gr

with gr.Blocks() as demo:
    gr.ChatInterface(collect_messages)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
